# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Question 2

### Retrieval of Toronto Neighborhoods Longitude and Latitude data

The following notebook is the answer of the second question in the Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto.



First we do the different step of question 1:

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
webcontent = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(webcontent,'lxml')
postaltable = soup.find('table')
postaltablecol = []
postaltablecontent = {}

for i, row in enumerate(postaltable.find_all('tr')):
    if i == 0:
        for Cval in row.find_all('th'):
            postaltablecol.append(Cval.text.replace('\n',''))
    else:
       postaltablecontent[i] =[row.find_all('td')[0].text ,  row.find_all('td')[1].text,  row.find_all('td')[2].text.replace('\n','')]

    
df = pd.DataFrame.from_dict(postaltablecontent,orient = 'index',columns = postaltablecol)
df.reset_index(inplace = True, drop = True)    
df = df[df['Borough']!= 'Not assigned']
df['Neighbourhood'].where(df['Neighbourhood'] != 'Not assigned',df['Borough'], inplace = True)
df = df.groupby(['Postcode','Borough']).Neighbourhood.unique().reset_index()
df['Neighbourhood']= df['Neighbourhood'].apply(', '.join)

df.head()



,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
df.shape

(103, 3)

Now we start the process of retrieving longitude and latitude information with the first example "M1B"

## I am not able to retrieve anything with the below code, and search indicate that there is a need for an api key; I will use the file provided insead.

In [6]:
import geocoder

In [ ]:

postal_code = 'M1G'
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

 ## Here the import from the downloaded file

In [28]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### I rename the Postal Code column to match my original df dataframe

In [35]:
df_geo.rename({'Postal Code': 'Postcode'},axis = 1, inplace = True)
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### I merge the 2 dataframe on the Postcode column to obtain the requested dataframe from question 2

In [36]:
df = pd.merge(df,df_geo,on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [37]:
df.shape

(103, 5)